In [4]:
import numpy as np
import random
import os
from sklearn.model_selection import KFold
from sklearn import svm
from sklearn.neural_network import MLPClassifier
# My libraries
import network
import fun

In [5]:
# 读取睡眠标签及观察值序列文件'..\\HMM_train\\hmm_obsv.txt',转换成network标准数据格式（x,y）
cwd = os.getcwd()
HMM_data_path = os.path.join(cwd, 'HMM_train')
file =  os.path.join(HMM_data_path, 'hmm_obsv.txt') 
# fid = open(file,"r")
# note = fid.readlines()
# fid.close()
with open(file,"r") as fid:
    note = fid.readlines()
    
tag = []
T = 960 
data = np.empty((len(note),T))
i = 0
for line in note:
    line_new = line.split(",")
    tag.append(fun.st2num(line_new[0][0]))
    data[i] = int(line_new[1])
    i+=1

#转换成（x,y）标准数据格式network 
data_net = [d.reshape(960,1) for d in data]
tag_net = [fun.vectorized_result(t) for t in tag]
     
data_tag = zip(data_net,tag_net)
data_tag = list(data_tag)

In [6]:
# len(data_tag)=4844
# 使用kfold（K-交叉验证）划分训练集、测试集
KF=KFold(n_splits=4) #建立4折交叉验证方法

for train_index,test_index in KF.split(data):
    print("TRAIN:\n",train_index,"\nTEST:\n",test_index)
    print("========")
    #data_train,data_test=data[train_index],data_tag[test_index]#报错，改成下面两行
    data_train = [data[index] for index in train_index]
    data_test = [data[index] for index in test_index]
    # 对应标签
    tag_train = [tag[index] for index in train_index]
    tag_test = [tag[index] for index in test_index]
    # 转换为sk_learn 标准数据格式
    data_train = (data_train, np.array(tag_train).reshape(len(tag_train),))
    data_test = (data_test, np.array(tag_test).reshape(len(tag_test),))
    
    # 1）使用sk-learn中的svm分类，做参照
    clf = svm.SVC()
    clf.fit(data_train[0], data_train[1])
    # test
    predictions = [int(a) for a in clf.predict(data_test[0])]
    num_correct = sum(int(a == y) for a, y in zip(predictions, data_test[1]))
    print("Baseline classifier using an SVM.")
    print(str(num_correct) + " of " + str(len(data_test[1])) + " values correct.")
    
    # 2）使用sk-learn中的MLPClassifier分类
    clf = MLPClassifier(alpha=1)
    clf.fit(data_train[0], data_train[1])
    # test
    predictions = [int(a) for a in clf.predict(data_test[0])]
    num_correct = sum(int(a == y) for a, y in zip(predictions, data_test[1]))
    print("Classifier using an MLP.")
    print(str(num_correct) + " of " + str(len(data_test[1])) + " values correct.")
    
    




TRAIN:
 [1211 1212 1213 ... 4841 4842 4843] 
TEST:
 [   0    1    2 ... 1208 1209 1210]
Baseline classifier using an SVM.
361 of 1211 values correct.
Classifier using an MLP.
231 of 1211 values correct.
TRAIN:
 [   0    1    2 ... 4841 4842 4843] 
TEST:
 [1211 1212 1213 ... 2419 2420 2421]
Baseline classifier using an SVM.
431 of 1211 values correct.
Classifier using an MLP.
552 of 1211 values correct.
TRAIN:
 [   0    1    2 ... 4841 4842 4843] 
TEST:
 [2422 2423 2424 ... 3630 3631 3632]
Baseline classifier using an SVM.
355 of 1211 values correct.
Classifier using an MLP.
343 of 1211 values correct.
TRAIN:
 [   0    1    2 ... 3630 3631 3632] 
TEST:
 [3633 3634 3635 ... 4841 4842 4843]
Baseline classifier using an SVM.
261 of 1211 values correct.
Classifier using an MLP.
422 of 1211 values correct.


In [15]:
# 结论：svm,hmm,network也很低，说明预处理中特征处理方式待改进，下面尝试bp-network
net = network.Network([960, 100, 6])
# use stochastic gradient descent to learn from the training_data over 30 epochs, 
# with a mini-batch size of 10, and a learning rate of η=3.0

# len(data_tag)=4844
# 使用kfold（K-交叉验证）划分训练集、测试集
KF=KFold(n_splits=4) #建立4折交叉验证方法

for train_index,test_index in KF.split(data_tag):
    print("TRAIN:\n",train_index,"\nTEST:\n",test_index)
    print("========")

    data_train = [data_tag[index] for index in train_index]
    data_test = [data_tag[index] for index in test_index]
#     print(data_test[0][1].shape)
#     print(data_test[0][0].shape)
#     print(len(data_test))
    net.SGD(data_train, 30, 100, 3, test_data=data_test)
    






TRAIN:
 [1211 1212 1213 ... 4841 4842 4843] 
TEST:
 [   0    1    2 ... 1208 1209 1210]
Epoch 0 : 49 / 1211
Epoch 1 : 49 / 1211
Epoch 2 : 536 / 1211
Epoch 3 : 63 / 1211
Epoch 4 : 66 / 1211
Epoch 5 : 536 / 1211
Epoch 6 : 536 / 1211
Epoch 7 : 63 / 1211
Epoch 8 : 360 / 1211
Epoch 9 : 536 / 1211
Epoch 10 : 536 / 1211
Epoch 11 : 66 / 1211
Epoch 12 : 125 / 1211
Epoch 13 : 360 / 1211
Epoch 14 : 516 / 1211
Epoch 15 : 360 / 1211
Epoch 16 : 360 / 1211
Epoch 17 : 360 / 1211
Epoch 18 : 152 / 1211
Epoch 19 : 536 / 1211
Epoch 20 : 360 / 1211
Epoch 21 : 360 / 1211
Epoch 22 : 360 / 1211
Epoch 23 : 66 / 1211
Epoch 24 : 172 / 1211
Epoch 25 : 360 / 1211
Epoch 26 : 172 / 1211
Epoch 27 : 360 / 1211
Epoch 28 : 152 / 1211
Epoch 29 : 172 / 1211
TRAIN:
 [   0    1    2 ... 4841 4842 4843] 
TEST:
 [1211 1212 1213 ... 2419 2420 2421]
Epoch 0 : 101 / 1211
Epoch 1 : 115 / 1211
Epoch 2 : 101 / 1211
Epoch 3 : 39 / 1211
Epoch 4 : 39 / 1211
Epoch 5 : 115 / 1211
Epoch 6 : 101 / 1211
Epoch 7 : 115 / 1211
Epoch 8 : 39 / 